# Imports, variables, functions

In [8]:
# -- IMPORTS --

import scipy.io
from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
import csv
import pickle
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
#from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator
import random
#from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_validate
#from imblearn.over_sampling import SMOTE
#from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from scipy import signal
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from plot_keras_history import plot_history

In [9]:
# -- ENVIRONMENT VARIABLES --

sample_rate = sampling_rate = 256
sec = 10
len_window = sample_rate*sec
overlap = 5
threshold = 2*sample_rate
sample_rate_downsample = int(0.1*sample_rate)
len_window_downsample = sample_rate_downsample*sec

patients = list(range(1, 80))
patient_with_issue = [4, 29, 50] 
for i in patient_with_issue:
    patients.remove(i)

# Load annotation file
annt = scipy.io.loadmat('../raw_data/annotations_2017.mat')

In [10]:
## -- PREPROCESSING FUNCTIONS --

# Highpass filter
def highpass_filter(signals, sampling_rate, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sampling_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

# Powerline filter
def notch_filter(signals, sampling_rate, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sampling_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

# Create our own scaler
class CustomTranformer(TransformerMixin, BaseEstimator): 
    # BaseEstimator generates the get_params() and set_params() methods that all Pipelines require
    # TransformerMixin creates the fit_transform() method from fit() and transform()
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.means = X.mean()
        return self
    
    def transform(self, X, y=None):
        norm_features = X - self.means
        return norm_features

# Combination of all filters and Scaler
def filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sampling_rate)
    filter_notch = notch_filter(filter_hp, sampling_rate, notch_frequency, quality_factor)
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

In [11]:
## -- LABEL FUNCTIONS --

# Format the EEG 
def eeg_formated(signals, names_ele):
    data_signals = signals.T # transpose the signals from datapoints
    data_signals = pd.DataFrame(data_signals) # create a pandas dataframe
    
    data_signals.columns = names_ele # rename columns
    
    return data_signals

# Format the annotations
def diagnosis(n):
    patient_A=annt["annotat_new"][0][n-1][0]
    patient_B=annt["annotat_new"][0][n-1][1]
    patient_C=annt["annotat_new"][0][n-1][2]
    
    #converting seconds to datapoints

    patient_A=patient_A.tolist()
    patient_B=patient_B.tolist()
    patient_C=patient_C.tolist()
    
    patient_A_dtp=[]
    patient_B_dtp=[]
    patient_C_dtp=[]  
    for elem in patient_A:
        for i in range(sampling_rate):
            patient_A_dtp.append(elem) 
    for elem in patient_B:
        for i in range(sampling_rate):
            patient_B_dtp.append(elem)
        
    for elem in patient_C:
        for i in range(sampling_rate):
            patient_C_dtp.append(elem)
            
    target_=pd.DataFrame({"Diagnosis A":patient_A_dtp,"Diagnosis B":patient_B_dtp,"Diagnosis C":patient_C_dtp})
    
    return target_  

# Create target variables when seizures lasts at least 10
def is_seizure(df):
    
    threshold = sampling_rate*10
    
    df['is_seizure_A'] = df["Diagnosis A"].groupby((df["Diagnosis A"] != df["Diagnosis A"].shift()).cumsum()).transform('size') * df["Diagnosis A"]
    df['is_seizure_A'] = (df['is_seizure_A'] > threshold).astype(int)
    
    df['is_seizure_B'] = df["Diagnosis B"].groupby((df["Diagnosis B"] != df["Diagnosis B"].shift()).cumsum()).transform('size') * df["Diagnosis B"]
    df['is_seizure_B'] = (df['is_seizure_B'] > threshold).astype(int)
    
    df['is_seizure_C'] = df["Diagnosis C"].groupby((df["Diagnosis C"] != df["Diagnosis C"].shift()).cumsum()).transform('size') * df["Diagnosis C"]
    df['is_seizure_C'] = (df['is_seizure_C'] > threshold).astype(int)
    
    return df 

# Create final target
def create_target(df):
    df['is_seizure_target'] = np.where(df['is_seizure_A'] + df['is_seizure_B'] + df['is_seizure_C'] >= 2, 1, 0)
    return df

# Remove useless
def remove_useless_columns(df):
    df.drop(columns=['Diagnosis A', 'Diagnosis B', 'Diagnosis C', 'is_seizure_A', 'is_seizure_B', 'is_seizure_C', 'ECG EKG', 'Resp Effort'], inplace=True)
    return df

# Final function to label
def label_data(path_raw_data, signals_preprocessed, n):
    
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])] # extract electrode names
    
    eeg_patient = eeg_formated(signals_preprocessed, names_ele) # format the ECG
    eeg_patient.rename(columns={'ECG EKG-REF':'ECG EKG', 'Resp Effort-REF':'Resp Effort'}, inplace=True)
    
    diagnosis_patient = diagnosis(n) # format the diagnosis
    
    data_patient = pd.merge(left=eeg_patient, right=diagnosis_patient, how='left', left_index=True, right_index=True) # merge ecg and diagnosis
    
    is_seizure(data_patient)
    create_target(data_patient)
    remove_useless_columns(data_patient)
    
    return data_patient

In [12]:
## -- FEATURE ENGINEERING --

def redefine_window_target(df,threshold=2):
    if len(np.unique(df.iloc[:,-1])) == 1:
        target = df.iloc[0,-1]
    elif np.unique(df.iloc[:,-1],return_counts=True)[1][1] >= threshold:
        target = 1
    else:
        target = 0
    t_df = df.drop(columns = "is_seizure_target")
    window = pd.DataFrame(np.array(t_df))
    window["Target"] = target
    return window

In [13]:
## -- MAIN FUNCTIONS --

def preprocess_and_label(path_raw_data, scaler, patient_number, Fournier=False):
    
    # Load raw data
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    # Preprocess data 
    signals_preprocessed = filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)
    
    if Fournier == True:
        signals_preprocessed = pd.DataFrame(np.array([abs(rfft(signals_preprocessed[i])) for i in range(len(signals_preprocessed))]))
        
    # Label data
    df = label_data(path_raw_data, signals_preprocessed, patient_number)
    
    return df

def prepare_x_y(df):
    #prepares the windows and x and y
    data=np.array([redefine_window_target(df.iloc[i:i+len_window ]) for i in range(0,len(df)-len_window, overlap*sample_rate)])
    X = data[:,:,:-1]
    y = data[:,:,-1]
    y=y.sum(axis=1)>=threshold*sample_rate
    #train and split indices 
    train_ind, test_ind = train_test_split(np.arange(X.shape[0]), test_size=10/X.shape[0])

    X_train = X[train_ind,:,:]
    X_test = X[test_ind,:,:]
    y_train = y[train_ind]
    y_test = y[test_ind]
    return X_train, X_test, y_train,y_test


# EEGs of unhealthy patients and Pipeline

In [14]:
patients = list(range(1, 80))

patient_with_issue = [4, 29, 50] # Can't import ECG4, ECG29 and ECG50
patient_without_seizure = [2, 3, 6, 8, 10, 12, 18, 24, 26, 27, 28, 30, 32, 35, 37, 42, 43, 45, 46, 48, 49, 53, 55, 56, 57, 58, 59, 60, 61,65, 70, 72]

for i in patient_with_issue:
    patients.remove(i)

for i in patient_without_seizure:
    patients.remove(i)

In [15]:
def model_for_each_patient(i, model):
    # Preprocess and label
    df_i = preprocess_and_label(f"../raw_data/eeg{i}.edf", CustomTranformer(), i, Fournier=False)
    
    # Downsample not needed for RNN
    #df_i_downsample = downsampling(df_i)
    
    # Flatten not needed for RNN
    #df_i_flat = flatten_dataframe(df_i_downsample)

    # Create x and y
    X_train, X_test, y_train, y_test =  prepare_x_y(df_i)
    
    #Fit model for each patient
    model.fit(X_train, y_train,
         batch_size=32,
         epochs=50, verbose=1)
    
    # Model evaluation
    results_DL =model.evaluate(X_test, y_test)
  
    # Mean of scores
    accuracy = results_DL[3]
    recall = results_DL[1]
    if results_DL[2]* results_DL[1]==0:
        f1=0
    else:
        f1 = 2*((results_DL[2]* results_DL[1])/(results_DL[2]+ results_DL[1]))
    
    return accuracy, recall, f1, y_train

# Train the model for all unhealthy patients

In [ ]:
model = Sequential()
model.add(layers.SimpleRNN(units=10, activation='tanh',input_shape=(2560, 19)))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# The compilation
model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy', 
        metrics=[ts.keras.metrics.Recall(),ts.keras.metrics.Precision(),"accuracy"])

accuracy_scores = []
recall_scores = []
precision_scores = [] 
f1_scores = []
dict_res={}
accuracy, recall,f1 ,y_train= model_for_each_patient(9, model)
accuracy_scores.append(accuracy)
recall_scores.append(recall)
f1_scores.append(f1)
dict_res[9]=[accuracy,recall,f1]

Epoch 1/50
22/22 [==============================] - 10s 402ms/step - loss: 0.3193 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - accuracy: 0.9928
Epoch 2/50
22/22 [==============================] - 9s 419ms/step - loss: 0.2471 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - accuracy: 1.0000
Epoch 3/50
22/22 [==============================] - 9s 420ms/step - loss: 0.1914 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - accuracy: 1.0000
Epoch 4/50
22/22 [==============================] - 8s 386ms/step - loss: 0.1471 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - accuracy: 1.0000
Epoch 5/50
22/22 [==============================] - 9s 397ms/step - loss: 0.1115 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - accuracy: 1.0000
Epoch 6/50
22/22 [==============================] - 9s 417ms/step - loss: 0.0841 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - accuracy: 1.0000
Epoch 7/50
22/22 [==============================] - 10s 438ms/step - loss: 0.0628 - recall_1: 0.0000e+00 

In [ ]:
y_train

In [ ]:
print(np.mean(accuracy_scores))
print(np.mean(recall_scores))
print(np.mean(precision_scores))
print(np.mean(f1_scores))

# Train model on patient 1 and save the fitted model

In [15]:
#Preprocess and label
df_1 = preprocess_and_label("../raw_data/eeg1.edf", CustomTranformer(), i, Fournier=False)
    
# Downsample
df_1_downsample = downsampling(df_1)
    
# Flatten
df_1_flat = flatten_dataframe(df_1_downsample)

# Train/test split
df_1_flat_train = df_1_flat.iloc[:int(0.7*len(df_1_flat)),:]
df_1_flat_test = df_1_flat.iloc[int(0.7*len(df_1_flat)):,:]

# X, y split
X_train, y_train = create_x_and_y(df_1_flat_train)
X_test, y_test = create_x_and_y(df_1_flat_test)

In [23]:
# Fit the model 
model = SVC(kernel='linear', C=0.001) 
model_fit = model.fit(X_train, y_train)

# Export the model 
#from joblib import dump, load
#dump(model_fit, 'model.joblib') 

In [25]:
model_fit

SVC(C=0.001, kernel='linear')

In [26]:
# Train accuracy
y_pred_train = model.predict(X_train)

train_score = accuracy_score(y_train, y_pred_train)
train_score

0.9836233367451381

In [29]:
# Test accuracy
y_pred_test = model.predict(X_test)

test_score = accuracy_score(y_test, y_pred_test)
test_score

0.7642857142857142